In [1]:
!pip install qiskit
!pip install qiskit_aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 78.3 MB/s eta 0:00:00


In [10]:
# Deutsch–Jozsa Algorithm using Qiskit 2.x
# Compatible with Qiskit >= 2.0.0

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [11]:
# ---------- ORACLES ----------
def oracle_constant(qc, ancilla, value=0):
    """
    Constant oracle: f(x)=0 or f(x)=1.
    If value is 1, it applies an X-gate to the ancilla.
    """
    if value == 1:
        qc.x(ancilla)


def oracle_balanced_parity(qc, inputs, ancilla):
    """
    Balanced oracle: f(x) = x0 XOR x1 XOR ... XOR xn.
    Implemented using CNOT gates from every input qubit to the ancilla.
    """
    for q in inputs:
        qc.cx(q, ancilla)

In [12]:
# ---------- DEUTSCH–JOZSA CIRCUIT ----------
def deutsch_jozsa_circuit(n, oracle_func, *oracle_args):
    """
    Constructs the Deutsch–Jozsa circuit.

    n: number of input qubits (excluding the ancilla)
    oracle_func: the oracle function to modify the circuit
    oracle_args: arguments required by the oracle function
    """
    # n+1 qubits (n inputs + 1 ancilla) and n classical bits for results
    qreg = QuantumRegister(n + 1, "q")
    creg = ClassicalRegister(n, "c")
    qc = QuantumCircuit(qreg, creg)

    inputs = list(range(n))
    ancilla = n  # The last qubit is the ancilla

    # Step 1: Initialize the ancilla qubit to |1>
    qc.x(ancilla)

    # Step 2: Apply Hadamard to all qubits
    qc.h(qreg)

    # Step 3: Apply the Oracle (Uf)
    oracle_func(qc, *oracle_args)

    # Step 4: Apply Hadamard to input qubits
    for q in inputs:
        qc.h(q)

    # Step 5: Measure only input qubits
    qc.measure(inputs, creg)

    return qc

In [13]:
# ---------- EXECUTION ----------
def run_dj(qc):
    """
    Run the Deutsch–Jozsa circuit on AerSimulator and analyze the results.
    """
    # Use the local Qiskit Aer Simulator
    simulator = AerSimulator()
    tqc = transpile(qc, simulator)
    job = simulator.run(tqc, shots=1024)
    result = job.result()
    counts = result.get_counts()

    print("Measurement counts:", counts)
    plot_histogram(counts)
    plt.show()

    n = qc.num_clbits
    # If all shots resulted in '0...0', the function is constant.
    if counts.get("0" * n, 0) == 1024:
        print("Function is CONSTANT (All results are |0...0>)")
    else:
        print("Function is BALANCED (At least one non-|0...0> result)")

In [14]:
# ---------- MAIN EXECUTION ----------
if __name__ == "__main__":
    n = 3  # Set the number of input qubits (2^n possible inputs)

    # --- Case 1: Constant Oracle (f(x)=0) ---
    print("\n=== Constant Oracle (f(x)=0) ===")
    qc_const = deutsch_jozsa_circuit(
        n, oracle_constant, n, 0  # ancilla is 'n', value is '0'
    )
    print("Circuit Diagram (Constant):\n", qc_const.draw(fold=-1))
    run_dj(qc_const)

    # --- Case 2: Balanced Oracle (Parity) ---
    print("\n=== Balanced Oracle (Parity) ===")
    qc_balanced = deutsch_jozsa_circuit(
        n, oracle_balanced_parity, list(range(n)), n  # inputs list, ancilla 'n'
    )
    print("Circuit Diagram (Balanced):\n", qc_balanced.draw(fold=-1))
    run_dj(qc_balanced)


=== Constant Oracle (f(x)=0) ===
Circuit Diagram (Constant):
      ┌───┐┌───┐┌─┐      
q_0: ┤ H ├┤ H ├┤M├──────
     ├───┤├───┤└╥┘┌─┐   
q_1: ┤ H ├┤ H ├─╫─┤M├───
     ├───┤├───┤ ║ └╥┘┌─┐
q_2: ┤ H ├┤ H ├─╫──╫─┤M├
     ├───┤├───┤ ║  ║ └╥┘
q_3: ┤ X ├┤ H ├─╫──╫──╫─
     └───┘└───┘ ║  ║  ║ 
c: 3/═══════════╩══╩══╩═
                0  1  2 
Measurement counts: {'000': 1024}
Function is CONSTANT (All results are |0...0>)

=== Balanced Oracle (Parity) ===
Circuit Diagram (Balanced):
      ┌───┐          ┌───┐     ┌─┐           
q_0: ┤ H ├───────■──┤ H ├─────┤M├───────────
     ├───┤       │  └───┘┌───┐└╥┘     ┌─┐   
q_1: ┤ H ├───────┼────■──┤ H ├─╫──────┤M├───
     ├───┤       │    │  └───┘ ║ ┌───┐└╥┘┌─┐
q_2: ┤ H ├───────┼────┼────■───╫─┤ H ├─╫─┤M├
     ├───┤┌───┐┌─┴─┐┌─┴─┐┌─┴─┐ ║ └───┘ ║ └╥┘
q_3: ┤ X ├┤ H ├┤ X ├┤ X ├┤ X ├─╫───────╫──╫─
     └───┘└───┘└───┘└───┘└───┘ ║       ║  ║ 
c: 3/══════════════════════════╩═══════╩══╩═
                               0       1  2 
Measurement counts: {'1

In [ ]:
#Task_1